[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nils-holmberg/scom-org/blob/main/jnb/socm_sa.ipynb)

# social media, nlp, text analysis

## load text data

In [17]:
import pandas as pd

# load fb post dataset
fp = "../csv/fb-sa-230314.csv"
fp = "https://raw.githubusercontent.com/nils-holmberg/scom-org/main/csv/fb-sa-230314.csv"
#dtp = read.table(fn, sep='\t', header=T, strip.white=TRUE, stringsAsFactors=FALSE)
#dtp = dtp |> mutate(month=month-30)
df = pd.read_csv(fp, header=0, sep="\t")
df.head()

,id,text,org,time,lang,follow,e_lc,e_cc,e_sc,e_rc,...,group,date,bins,index,month,sa_int_abs,sa_int,wc,out,org_type
0,PlanSverige-10154245440447004,Hur är det att föda barn i ett land som drabba...,PlanSverige,1.446360e+09,sv,37539,525,10,21,0,...,25,2015-11-01,201511,2588,31,3,9,61,1,9
1,oxfamsverige-654404778032860,Lilla Jana 16 månader är en av miljontals männ...,oxfamsverige,1.446365e+09,sv,8838,103,0,11,0,...,25,2015-11-01,201511,2589,31,6,36,80,0,9
2,hallsverigerent-10153131904127477,Heja Smögens fiskauktion och havochvatten! Til...,hallsverigerent,1.446366e+09,sv,11288,192,6,16,0,...,25,2015-11-01,201511,2590,31,2,4,22,1,5
3,ClownerutanGranser-995657940490841,Ikväll gästar vår vän Suzanne Reuter även känd...,ClownerutanGranser,1.446367e+09,sv,7449,219,2,10,0,...,25,2015-11-01,201511,2591,31,3,9,51,0,9
4,lakareutangranser-10153338180335787,Den 8:e november är det Fars dag! Fira med en ...,lakareutangranser,1.446408e+09,sv,364399,510,1,45,0,...,25,2015-11-01,201511,2594,31,6,16,45,0,3


In [18]:
df['lang'].value_counts()

sv    90586
en     6602
Name: lang, dtype: int64

In [19]:
df.loc[995:1005,["id","text"]]

,id,text
995,SLMK.1981-1238466769502481,Vi måste lära oss av vår historia.
996,raddningsmissionen-10153378639580345,Nu är julklappsinsamlingen på Gothia Towers i ...
997,GbgStadsmission-1107919595899887,Göteborgs Stadsmissions julfilm sätter i år fo...
998,svenskalivraddningssallskapet-902883429759128,Nu börjar isarna lägga sig. Härliga dagar med ...
999,civilrightsdefenders-916451015059425,ETHIOPIA UPDATE: A few months after being rele...
1000,rodakorset-1079984975365462,Just nu diskuteras det mycket hur vi ska ta ha...
1001,Erikshjalpen-1052622464768598,Nu har vi fyllt på med nya Dece-prylar i gåvob...
1002,HRFriks-912182902170613,”Hur mår svensk funktionshinderforskning?” är ...
1003,sjoraddning-10153849707263939,Våra sjöräddare från RS Skillinge genomförde u...
1004,GbgStadsmission-1107957849229395,Det börjar bli julstämning i Göteborg... och s...


## sentence tokenize

In [25]:
import nltk
from nltk.tokenize import sent_tokenize
#nltk.download('popular')

# Tokenize the text into sentences and create a new DataFrame
sentences_df = pd.DataFrame(columns=['id', 'sentence_number', 'sentence'])

for idx, row in df.loc[995:1005,["id","text"]].iterrows():
    text_id = row['id']
    text = row['text']
    sentences = sent_tokenize(text)

    # Create a DataFrame for sentences in the current text
    sentences_data = {
        'id': [text_id] * len(sentences),
        'sentence_number': list(range(1, len(sentences) + 1)),
        'sentence': sentences
    }

    # Concatenate the current text's sentences with the overall sentences DataFrame
    sentences_df = pd.concat([sentences_df, pd.DataFrame(sentences_data)], ignore_index=True)

# Print the final DataFrame
sentences_df


,id,sentence_number,sentence
0,SLMK.1981-1238466769502481,1,Vi måste lära oss av vår historia.
1,raddningsmissionen-10153378639580345,1,Nu är julklappsinsamlingen på Gothia Towers i ...
2,raddningsmissionen-10153378639580345,2,För väldigt många barn i Göteborg är julklappa...
3,raddningsmissionen-10153378639580345,3,Därför finns Göteborgs viktigaste julklappsins...
4,raddningsmissionen-10153378639580345,4,I fjol samlades det in över 10 500 klappar til...
5,raddningsmissionen-10153378639580345,5,Julklapparna som samlas in i år går i första h...
6,raddningsmissionen-10153378639580345,6,Förutom Räddningsmissionens egna verksamheter ...
7,raddningsmissionen-10153378639580345,7,Vill du vara med & ge en julklapp?
8,raddningsmissionen-10153378639580345,8,Vi önskar nya klappar & att de är oinslagna.
9,raddningsmissionen-10153378639580345,9,Detta för att verkligen kunna ge barnen det so...


In [21]:
# Specify your desired output file path
fp = 'fb-sa-231228.tsv'

# Save the DataFrame as a TSV file
sentences_df.to_csv(fp, sep='\t', index=False)

sentences_df = pd.read_csv(fp, header=0, sep="\t")
sentences_df.head()

## sentiment analysis

In [22]:
!pip install -q transformers


### untrained model

In [28]:
from transformers import pipeline

# Perform sentiment analysis using Hugging Face Transformers
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-multilingual-cased")

sentiment_analyzer("i love this movie!")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_0', 'score': 0.5471532344818115}]

In [ ]:
# Iterate through each sentence and analyze sentiment
sentences_df['sentiment'] = sentences_df['sentence'].apply(lambda x: sentiment_analyzer(x)[0])

sentences_df.head()


### trained model

In [29]:
from transformers import pipeline

sentiment_analyzer = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    return_all_scores=True
)

# english
sentiment_analyzer("i love this movie!")


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'positive', 'score': 0.9912461638450623},
  {'label': 'neutral', 'score': 0.0058813379146158695},
  {'label': 'negative', 'score': 0.0028724849689751863}]]

In [30]:
# swedish
sentiment_analyzer("jag älskar den här filmen!")


[[{'label': 'positive', 'score': 0.7703008651733398},
  {'label': 'neutral', 'score': 0.061975058168172836},
  {'label': 'negative', 'score': 0.16772407293319702}]]

In [31]:
# Iterate through each sentence and analyze sentiment
sentences_df['sentiment'] = sentences_df['sentence'].apply(lambda x: sentiment_analyzer(x)[0])

sentences_df.head()

,id,sentence_number,sentence,sentiment
0,SLMK.1981-1238466769502481,1,Vi måste lära oss av vår historia.,"[{'label': 'positive', 'score': 0.586677789688..."
1,raddningsmissionen-10153378639580345,1,Nu är julklappsinsamlingen på Gothia Towers i ...,"[{'label': 'positive', 'score': 0.526108324527..."
2,raddningsmissionen-10153378639580345,2,För väldigt många barn i Göteborg är julklappa...,"[{'label': 'positive', 'score': 0.217555463314..."
3,raddningsmissionen-10153378639580345,3,Därför finns Göteborgs viktigaste julklappsins...,"[{'label': 'positive', 'score': 0.575258433818..."
4,raddningsmissionen-10153378639580345,4,I fjol samlades det in över 10 500 klappar til...,"[{'label': 'positive', 'score': 0.473267465829..."
